In [3]:
import sys
from nbfinder import NotebookFinder
sys.meta_path.append(NotebookFinder())
import tensorflow as tf
if __name__ == "__main__":
    sys.path.append("../../")
from notebooks_src.losses.util import unpack_net_output, sort_some_lists_of_tensors
from notebooks_src.box_encode_decode.ssd.encode import encode
from notebooks_src.box_encode_decode.ssd.decode import decode
from notebooks_src.metrics.configs import configs
from notebooks_src.load_data.configs import configs as data_configs
from notebooks_src.postprocessing.ssd import detected_bboxes
configs.update(data_configs)

In [10]:
def calc_accuracy(y_true, y_preds):
    gclasses, glocalizations, gscores = encode(y_true)


    localizations, logits, predictions = unpack_net_output(y_preds)
    
    logits, localizations, gclasses, glocalizations, gscores, predictions = sort_some_lists_of_tensors(logits,
                                                                                           localizations, 
                                                                                           gclasses, 
                                                                                           glocalizations, 
                                                                                           gscores, predictions)

    localizations = decode(localizations)
    
    # get top k  predicted boxes after nms
    rscores, rbboxes = detected_bboxes(predictions, localizations,
                            select_threshold=configs["select_threshold"],
                            nms_threshold=configs["nms_threshold"],
                            clipping_bbox=None,
                            top_k=configs["select_top_k"],
                            keep_top_k=configs["keep_top_k"])
    return rscores, rboxes
    
    

In [12]:
if __name__ == "__main__":
    with tf.Session() as sess:
        from notebooks_src.load_data.get_generator import get_generator

        gen=get_generator("tr", batch_size=2)
        y_true = tf.placeholder(tf.float32,shape=(2,15,5),name="y_true")
        shapes = [(2, 6, 9, 48),
                 (2, 3, 5, 32),
                 (2, 96, 144, 32),
                 (2, 24, 36, 48),
                 (2, 12, 18, 48),
                 (2, 48, 72, 48),
                 (2, 1, 1, 32)]

        y_preds = [tf.ones((shape)) for shape in shapes]
        rs,rb = calc_accuracy(y_true, y_preds)
#         for im, box in gen:
#             print sess.run(final_loss, feed_dict={bboxes:box})

TypeError: detected_bboxes() takes at least 3 arguments (7 given)

In [4]:
localizations

[<tf.Tensor 'ssd_bboxes_decode/stack:0' shape=(2, 96, 144, 4, 4) dtype=float32>,
 <tf.Tensor 'ssd_bboxes_decode/stack_1:0' shape=(2, 48, 72, 6, 4) dtype=float32>,
 <tf.Tensor 'ssd_bboxes_decode/stack_2:0' shape=(2, 24, 36, 6, 4) dtype=float32>,
 <tf.Tensor 'ssd_bboxes_decode/stack_3:0' shape=(2, 12, 18, 6, 4) dtype=float32>,
 <tf.Tensor 'ssd_bboxes_decode/stack_4:0' shape=(2, 6, 9, 6, 4) dtype=float32>,
 <tf.Tensor 'ssd_bboxes_decode/stack_5:0' shape=(2, 3, 5, 4, 4) dtype=float32>,
 <tf.Tensor 'ssd_bboxes_decode/stack_6:0' shape=(2, 1, 1, 4, 4) dtype=float32>]

In [ ]:

batch_shape = [1] * 5 + [len(ssd_anchors)] * 3


# Make a batch of data
r = tf.train.batch(
    tf_utils.reshape_list([image, glabels, gbboxes, gdifficults, gbbox_img,
                           gclasses, glocalisations, gscores]),
    batch_size=FLAGS.batch_size,
    num_threads=FLAGS.num_preprocessing_threads,
    capacity=5 * FLAGS.batch_size,
    dynamic_pad=True)
(b_image, b_glabels, b_gbboxes, b_gdifficults, b_gbbox_img, b_gclasses,
 b_glocalisations, b_gscores) = tf_utils.reshape_list(r, batch_shape)

# =================================================================== #
# SSD Network + Ouputs decoding.
# =================================================================== #
dict_metrics = {}
arg_scope = ssd_net.arg_scope(data_format=DATA_FORMAT)
# 1. get outputs from network
# localisations is the box corrdinates
# logits is the raw output of the convolution for classes (ie unnormalized prob dist)
# predictions is the softmax of the logits
predictions, localisations, logits, end_points = \
    ssd_net.net(b_image, is_training=False)


# Performing post-processing on CPU: loop-intensive, usually more efficient.
with tf.device('/device:CPU:0'):
    
    
# decode encoded output for normalization to box coords?
localisations = ssd_net.bboxes_decode(localisations, ssd_anchors)


# get top k  predicted boxes after nms
rscores, rbboxes = \
    ssd_net.detected_bboxes(predictions, localisations,
                            select_threshold=FLAGS.select_threshold,
                            nms_threshold=FLAGS.nms_threshold,
                            clipping_bbox=None,
                            top_k=FLAGS.select_top_k,
                            keep_top_k=FLAGS.keep_top_k)
    
    
#match the predicted boxes to ground truth boxes and compute the TP and FP statistics.
num_gbboxes, tp, fp, rscores = \
    tfe.bboxes_matching_batch(rscores.keys(), rscores, rbboxes,
                              b_glabels, b_gbboxes, b_gdifficults,
                              matching_threshold=FLAGS.matching_threshold)

In [ ]:
tp_fp_metric = tfe.streaming_tp_fp_arrays(num_gbboxes, tp, fp, rscores)
            for c in tp_fp_metric[0].keys():
                dict_metrics['tp_fp_%s' % c] = (tp_fp_metric[0][c],
                                                tp_fp_metric[1][c])

            # Add to summaries precision/recall values.
            aps_voc07 = {}
            aps_voc12 = {}
            for c in tp_fp_metric[0].keys():
                # Precison and recall values.
                prec, rec = tfe.precision_recall(*tp_fp_metric[0][c])

                # Average precision VOC07.
                v = tfe.average_precision_voc07(prec, rec)
                summary_name = 'AP_VOC07/%s' % c
                op = tf.summary.scalar(summary_name, v, collections=[])
                # op = tf.Print(op, [v], summary_name)
                tf.add_to_collection(tf.GraphKeys.SUMMARIES, op)
                aps_voc07[c] = v

                # Average precision VOC12.
                v = tfe.average_precision_voc12(prec, rec)
                summary_name = 'AP_VOC12/%s' % c
                op = tf.summary.scalar(summary_name, v, collections=[])
                # op = tf.Print(op, [v], summary_name)
                tf.add_to_collection(tf.GraphKeys.SUMMARIES, op)
                aps_voc12[c] = v

            # Mean average precision VOC07.
            summary_name = 'AP_VOC07/mAP'
            mAP = tf.add_n(list(aps_voc07.values())) / len(aps_voc07)
            op = tf.summary.scalar(summary_name, mAP, collections=[])
            op = tf.Print(op, [mAP], summary_name)
            tf.add_to_collection(tf.GraphKeys.SUMMARIES, op)

            # Mean average precision VOC12.
            summary_name = 'AP_VOC12/mAP'
            mAP = tf.add_n(list(aps_voc12.values())) / len(aps_voc12)
            op = tf.summary.scalar(summary_name, mAP, collections=[])
            op = tf.Print(op, [mAP], summary_name)
            tf.add_to_collection(tf.GraphKeys.SUMMARIES, op)


In [ ]:
class streaming_acc_class(object):
    def __init__(self, inp mode="VOC"):
        #intialize tensors for calculating one batch of acc
        pass
        
    def update_acc(self,x,y):
        #do a forward pass with feed dict and get tp/fp for each class and append to the master ones
        pass
    def get_final_acc(self):
        # do final prec_recall on each class and return the average as well
        
        pass